## Open Government Data, provided by **OpenDataZurich**
*Autogenerated Python starter code for data set with identifier* **geo_standorte_der_automatischen_fuss__und_velozaehlungen**

## Dataset
# **Standorte der automatischen Fuss- und Velozählungen**

## Description

Das Tiefbauamt erhebt seit 2009 die Velofrequenzen in der Stadt Zürich mit Hilfe automatischer Zählgeräte. Der Fussverkehr wird seit 2012 mit Geräten gezählt. Das Zählstellennetz wurde schrittweise gemäss einem definierten Standortkonzept ausgebaut und umfasst jeweils rund 20 - 25 Querschnitten im gesamten Stadtgebiet. Die damit erhobenen Daten bilden eine wichtige quantitative Grundlage zur Beurteilung der Veloförderung im Sinne von «Stadtverkehr 2025» und dem Masterplan Velo. Die Daten liegen richtungsgetrennt und auf Viertelstundenwerte aggregiert vor. Die Daten stellen Gerätewerte dar. Um die effektiven Frequenzen im Fuss- oder Veloverkehr zu bestimmen, müssen die angegebenen Korrekturfaktoren angewandt werden.
In diesem Datensatz sind nur die eigentlichen Standorte enthalten. Die dazugehörenden Zählwerte sind über Open Data Zürich erhältlich (https://data.stadt-zuerich.ch/dataset/ted_taz_verkehrszaehlungen_werte_fussgaenger_velo).

**Zweck**: Die erhobenen Daten werden für das Controlling von Stadtverkehr 2025 und Masterplan Velo benötigt. Sie bilden zudem eine wichtige Grundlage zur Dokumentation der Bedeutungen des Fuss- und Veloverkehrs und für die Erfolgskontrolle deren Förderung.

**Genereller Hinweis zum Geodatensatz:**


Weitere Informationen zu diesem Datensatz finden sie unter [«**Standorte der automatischen Fuss- und Velozählungen»** auf Geocat.ch](https://www.geocat.ch/geonetwork/srv/ger/catalog.search#/metadata/13bd8bea-446e-47a7-b6de-961ebd5d1fec).				
			   

**Datenerfassung:**

Automatische Velozählung: Die Daten der permanenten Velozählstellen informieren über die Anzahl der Velofahrten über die im Strassenquerschnitt eingebauten Zählschlaufen. Auch im gemischten Verkehr werden gemäss Herstellerangaben unabhängig von der Verkehrsmenge ausschliesslich Velos gezählt. Das System wurde speziell dafür entwickelt, motorisierte Fahrzeuge, einschliesslich Motorräder, bei der Zählung auszuschliessen, indem diese aufgrund eines patentierten Algorithmus ignoriert werden. Velos mit Karbonfelgen werden ebenfalls nicht gezählt. Regelmässige manuelle Kontrollzählungen werden durchgeführt, um die Genauigkeit der Zählgeräte zu überprüfen.

Automatische Zählungen des Fussverkehrs: Die Daten der Zählgeräte informieren über die Anzahl der Bewegungen im vom Zählgerät erfassten Querschnitt mittels passiver Infrarotsensoren. Die Geräte sind so angebracht, dass die erfassten Daten die Anzahl der Passanten ausweisen. Regelmässige manuelle Kontrollzählungen werden durchgeführt, um die Genauigkeit der Zählgeräte zu überprüfen. Es gibt Querschnitte, an denen zusätzlich zu den Fussfrequenzen auch Velozählungen stattfinden, um die Differenz beider Werte bilden zu können (Differenz entspricht den effektiven Fussfrequenzen).

Einzelne Zählgeräte erfassen den Fuss- und den Veloverkehr parallel und können beide Verkehrsarten unterscheiden.

**Datengrundlage:**

Alle Daten werden anonym erhoben, Rückschlüsse auf die Verkehrsteilnehmenden sind nicht möglich. Die Zähldaten werden zu 15-Minuten-Intervallen richtungsgetrennt aggregiert und täglich drahtlos übertragen.

**Weitere Informationen zu diesem Thema:**

Finden Sie unter diesem [**Link**](https://www.stadt-zuerich.ch/artikel/de/statistik-und-daten/automatische-zaehlungen-des-veloverkehrs.html)

**Statisches Vorschaubild:**

![BildText](https://www.gis.stadt-zuerich.ch/zueriplan_docs/geocat/13bd8bea-446e-47a7-b6de-961ebd5d1fec.png)



## Data set links

[Direct link by OpenDataZurich for dataset](https://data.stadt-zuerich.ch/dataset/geo_standorte_der_automatischen_fuss__und_velozaehlungen)

https://www.stadt-zuerich.ch/geodaten/download/Standorte_der_automatischen_Fuss__und_Velozaehlungen?format=geojson_link<br>


## Metadata
- **Publisher** `Grundlagen + Strategien, Tiefbauamt, Tiefbau- und Entsorgungsdepartement`
- **Maintainer** `Open Data Zürich`
- **Maintainer_email** `opendata@zuerich.ch`
- **Keywords** `Mobilität`
- **Tags** `['frequenzen', 'fussverkehr', 'geodaten', 'geoportal', 'punktdaten', 'stzh', 'vektordaten', 'veloverkehr', 'verkehrszahlung']`
- **Metadata_created** `2023-11-06T03:10:36.069239`
- **Metadata_modified** `2025-10-25T06:33:31.634650`


## Imports and helper functions

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd 

import json
import xml.etree.ElementTree as ET
import requests
import geopandas as gpd
import re

In [ ]:
# helper function for reading datasets with proper separator
def get_dataset(url):
    # get URL ofs WFS service
    dataset_identifier = re.search(r'\/([^\/\?]+)\?', url).group(1)
    url_geoportal = f"https://www.ogd.stadt-zuerich.ch/wfs/geoportal/{dataset_identifier}"
    print("Getting available layers from:", url_geoportal)
    
    # Parameter for GetCapabilities
    params = {
        "service": "WFS",
        "version": "1.1.0",
        "request": "GetCapabilities"
    }
    
    # send GetCapabilities
    response = requests.get(url_geoportal, params=params)

    # parse XML answer
    root = ET.fromstring(response.content)

    # define Namespace 
    namespace = {'wfs': 'http://www.opengis.net/wfs'}
    
    # Exctract available layers
    layers = [feature_type.find('wfs:Name', namespace).text for feature_type in root.findall('.//wfs:FeatureType', namespace)]
    
    print("Available layers:", layers)
    print("First layer is set as default. To chose another layer set it as typename in the get_dataset() function.")

    # set first layer als typename
    typename = layers[0]
    print("Chosen typename:", typename)

    # Parameter GetFeature request
    params = {
        "service": "WFS",
        "version": "1.1.0",
        "request": "GetFeature",
        "typename": typename,
        "outputFormat": "application/json"
    }
    
    # GetFeature request
    response = requests.get(url_geoportal, params=params)

    # Load GeoJSON in GeoDataFrame
    gdf = gpd.read_file(json.dumps(response.json()))
    return gdf

## Load the data

In [ ]:
gdf = get_dataset('https://www.stadt-zuerich.ch/geodaten/download/Standorte_der_automatischen_Fuss__und_Velozaehlungen?format=geojson_link')

## Analyze the data

In [ ]:
gdf.plot()

In [ ]:
# drop columns that have no values
gdf.dropna(how='all', axis=1, inplace=True)

In [ ]:
print(f'The dataset has {gdf.shape[0]:,.0f} rows (observations) and {gdf.shape[1]:,.0f} columns (variables).')
print(f'There seem to be {gdf.duplicated().sum()} exact duplicates in the data.')

In [ ]:
gdf.info(memory_usage='deep', verbose=True)

In [ ]:
gdf.head()

In [ ]:
# display a small random sample transposed in order to see all variables
gdf.sample(3).T

In [ ]:
# describe non-numerical features
try:
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        display(gdf.describe(exclude='number'))
except:
    print("No categorical data in dataset.")

In [ ]:
# describe numerical features
try:
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        display(gdf.describe(include='number'))
except:
    print("No numercial data in dataset.")

In [ ]:
# plot a histogram for each numerical feature
try:
    gdf.hist(bins=25, rwidth=.9)
    plt.tight_layout()
    plt.show()
except:
    print("No numercial data to plot.") 

In [ ]:
# continue your code here...

**Contact**: opendata@zuerich.ch